In [1]:
import pandas as pd
import numpy as np

from pycta.portfolio import Portfolio
from pycta.beakerx import *

In [2]:
# Load prices
prices = pd.read_csv("data/prices.csv", index_col=0, parse_dates=True).ffill().truncate(before="1970-01-01")

In [3]:
# take two moving averages and apply the sign-function, adjust by volatility
def f(price, fast=32, slow=96, volatility=32):
    s = price.ewm(com=slow, min_periods=300).mean()
    f = price.ewm(com=fast, min_periods=300).mean()
    std = price.pct_change().ewm(com=volatility, min_periods=300).std()
    return np.sign(f-s)/std

In [6]:
form = EasyForm("Configuration")
form.addTextField("fast")
form.addTextField("slow")
form.addTextField("volatility")
form.addButton("Go!", tag="run")
form


In [9]:
portfolio = Portfolio(prices=prices, position=prices.apply(f, fast=int(form["fast"] or 32), slow=int(form["slow"] or 96), volatility=int(form["volatility"] or 96)))
analysis(portfolio.nav())

* This is a **univariate** trading system, we map the (real) price of an asset to its (cash)position
* Only 3 **free parameters** used here.
* Only 4 lines of code
* Scaling the bet-size by volatility has improved the situation.

Results do not look terrible but...
* No concept of risk integrated 

Often hedge funds outsource the risk management to some board or committee
and develop machinery for more systematic **parameter-hacking**.